In [1]:
using Random, Distributions, DataFrames, GLM, Statistics

In [2]:
α = 1.78
β = 1
Q = 100 # Liczba państw

100

In [3]:
p = rand(Uniform(.1,.75), Q)

N = reduce(vcat, rand.(Binomial.(rand(DiscreteUniform(100, 1000), Q), .1), 1))
M = reduce(vcat, rand.(Poisson.(N .^ α), 1))

m = reduce(vcat, rand.(Poisson.(M .* p), 1))
n = reduce(vcat, rand.(Binomial.(N, p), 1))

100-element Vector{Int64}:
 19
  7
 48
 11
 16
 11
 44
 33
 31
 23
 64
 11
 62
  ⋮
  8
 48
 19
 79
 16
  9
 37
 31
 22
  8
 31
 12

In [4]:
df = DataFrame(
    y = log.(m ./ N),
    x1 = log.(N),
    x2 = log.(n ./ N)
)

Row,y,x1,x2
,Float64,Float64,Float64
1,2.11678,3.13549,-0.191055
2,1.32687,2.56495,-0.619039
3,2.79865,4.26268,-0.391479
4,1.44465,4.64439,-2.2465
5,1.61498,3.58352,-0.81093
6,1.72722,3.17805,-0.780159
7,2.76272,4.2485,-0.464306
8,2.42203,4.20469,-0.708185
9,2.66259,4.56435,-1.13036


In [5]:
df1 = DataFrame(
    y = log.(m),
    x1 = log.(N),
    x2 = log.(n ./ N)
)
lm(@formula(y ~ x1 + x2 + 0), df1)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

y ~ 0 + x1 + x2

Coefficients:
────────────────────────────────────────────────────────────────
       Coef.  Std. Error       t  Pr(>|t|)  Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────
x1  1.74638   0.00954512  182.96    <1e-99   1.72744    1.76532
x2  0.840012  0.0333134    25.22    <1e-43   0.773903   0.906121
────────────────────────────────────────────────────────────────

In [6]:
ols = lm(@formula(y ~ x1 + x2 + 0), df)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

y ~ 0 + x1 + x2

Coefficients:
───────────────────────────────────────────────────────────────
       Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────
x1  0.746382  0.00954512  78.20    <1e-89   0.72744    0.765324
x2  0.840012  0.0333134   25.22    <1e-43   0.773903   0.906121
───────────────────────────────────────────────────────────────

In [7]:
α̂, β̂ = coef(ols) + [1;0]

2-element Vector{Float64}:
 1.7463820016477956
 0.8400120071749972

In [8]:
ξ̂ = N .^ α̂

100-element Vector{Float64}:
  238.83445737385026
   88.18010696938116
 1710.0320911092897
 3330.5098350612243
  522.274325011522
  257.26225436302934
 1668.1919008982907
 1545.3404375161551
 2896.02831527883
  863.1562854360719
 3055.915812647661
 2688.5846621047253
 1881.799038843499
    ⋮
  683.6177726240803
 3164.541334436326
  238.83445737385026
 3330.5098350612243
  992.652759185115
  448.6476671276475
 1060.281739509783
  712.2950250709482
 1094.808289998648
  926.9378941428532
  771.1530470610775
 2438.605307143071

In [9]:
@show [sum(M), sum(N .^ α), sum(ξ̂)]

[sum(M), sum(N .^ α), sum(ξ̂)] = [150295.0, 150458.91853372223, 130324.89260397527]


3-element Vector{Float64}:
 150295.0
 150458.91853372223
 130324.89260397527

In [10]:
df1 = DataFrame(
    y = m,
    x1 = log.(N),
    x2 = log.(n ./ N)
)

mm = glm(@formula(y ~ x1 + x2 + 0), df1, Poisson(), LogLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Poisson{Float64}, LogLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

y ~ 0 + x1 + x2

Coefficients:
─────────────────────────────────────────────────────────────────
       Coef.  Std. Error        z  Pr(>|z|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────
x1  1.76485   0.00159636  1105.55    <1e-99   1.76172    1.76798
x2  0.913676  0.00841233   108.61    <1e-99   0.897188   0.930164
─────────────────────────────────────────────────────────────────

In [11]:
α̂₁, β̂₁ = coef(mm)

2-element Vector{Float64}:
 1.7648532100084853
 0.9136762237970035

In [12]:
@show [sum(M), sum(N .^ α), sum(ξ̂), sum(N .^ α̂₁)]

[sum(M), sum(N .^ α), sum(ξ̂), sum(N .^ α̂₁)] = [150295.0, 150458.91853372223, 130324.89260397527, 141025.928203878]


4-element Vector{Float64}:
 150295.0
 150458.91853372223
 130324.89260397527
 141025.928203878

In [13]:
α = 1.78
β = 1
Q = 100 # Liczba państw

res = DataFrame(
    actual     = [],
    expected   = [],
    glm_est    = [],
    ols_est    = []
)

for k in 1:1000
    N = reduce(vcat, rand.(Binomial.(rand(DiscreteUniform(2000, 4000), Q), .1), 1))
    M = reduce(vcat, rand.(Poisson.(N .^ α), 1))
    p = rand(Uniform(.1,.75), Q)
    
    m = reduce(vcat, rand.(Poisson.(M .* p), 1))
    n = reduce(vcat, rand.(Binomial.(N, p), 1))

    df1 = DataFrame(
        y = m,
        x1 = log.(N),
        x2 = log.(n ./ N)
    )

    mm = glm(@formula(y ~ x1 + x2 + 0), df1, Poisson(), LogLink())
    α̂₁, β̂₁ = coef(mm)

    ols = lm(@formula(log(y) ~ x1 + x2 + 0), df1)
    α̂₂, β̂₂ = coef(ols)
    
    push!(res, [sum(M) sum(N .^ α) sum(N .^ α̂₁) sum(N .^ α̂₂)])
end # end for

In [14]:
DataFrame(
    rel_bias = [mean((res[:, 1] .- res[:, 2]) ./ res[:, 1]), mean((res[:, 1] .- res[:, 3]) ./ res[:, 1]), mean((res[:, 1] .- res[:, 4]) ./ res[:, 1])],
    rel_mse  = [mean(((res[:, 1] .- res[:, 2]) .^ 2) ./ res[:, 1]), mean(((res[:, 1] .- res[:, 3]) .^ 2) ./ res[:, 1]), mean(((res[:, 1] .- res[:, 4]) .^ 2) ./ res[:, 1])],
    rel_mae  = [mean(abs.(res[:, 1] .- res[:, 2]) ./ res[:, 1]), mean(abs.(res[:, 1] .- res[:, 3]) ./ res[:, 1]), mean(abs.(res[:, 1] .- res[:, 4]) ./ res[:, 1])],
    est  = ["Expected value","glm","ols"]
)

Row,rel_bias,rel_mse,rel_mae,est
,Float64,Float64,Float64,String
1,-1.69215e-5,1.00738,0.000495101,Expected value
2,0.0179668,1261.93,0.0189396,glm
3,0.0271943,2834.57,0.0283322,ols


In [15]:
DataFrame(
    rel_bias = [mean((res[:, 2] .- res[:, 3]) ./ res[:, 2]), mean((res[:, 2] .- res[:, 4]) ./ res[:, 2])],
    rel_mse  = [mean(((res[:, 2] .- res[:, 3]) .^ 2) ./ res[:, 2]), mean(((res[:, 2] .- res[:, 4]) .^ 2) ./ res[:, 2])],
    rel_mae  = [mean(abs.(res[:, 2] .- res[:, 3]) ./ res[:, 2]), mean(abs.(res[:, 2] .- res[:, 4]) ./ res[:, 2])],
    est  = ["glm","ols"]
)

Row,rel_bias,rel_mse,rel_mae,est
,Float64,Float64,Float64,String
1,0.0179829,1265.21,0.0189673,glm
2,0.0272103,2838.43,0.0283473,ols


In [16]:
α = 2.3
β = 1.2
Q = 20 # Liczba państw

res = DataFrame(
    actual     = [],
    expected   = [],
    glm_est    = [],
    ols_est    = []
)

for k in 1:1000
    N = reduce(vcat, rand.(Binomial.(rand(DiscreteUniform(200, 400), Q), .3), 1))
    M = reduce(vcat, rand.(Poisson.(N .^ α), 1))
    p = rand(Uniform(.1,.75), Q)
    
    m = reduce(vcat, rand.(Poisson.(M .* p), 1))
    n = reduce(vcat, rand.(Binomial.(N, p), 1))

    df1 = DataFrame(
        y = m,
        x1 = log.(N),
        x2 = log.(n ./ N)
    )

    mm = glm(@formula(y ~ x1 + x2 + 0), df1, Poisson(), LogLink())
    α̂₁, β̂₁ = coef(mm)

    ols = lm(@formula(log(y) ~ x1 + x2 + 0), df1)
    α̂₂, β̂₂ = coef(ols)
    
    push!(res, [sum(M) sum(N .^ α) sum(N .^ α̂₁) sum(N .^ α̂₂)])
end # end for

DataFrame(
    rel_bias = [mean((res[:, 1] .- res[:, 2]) ./ res[:, 1]), mean((res[:, 1] .- res[:, 3]) ./ res[:, 1]), mean((res[:, 1] .- res[:, 4]) ./ res[:, 1])],
    rel_mse  = [mean(((res[:, 1] .- res[:, 2]) .^ 2) ./ res[:, 1]), mean(((res[:, 1] .- res[:, 3]) .^ 2) ./ res[:, 1]), mean(((res[:, 1] .- res[:, 4]) .^ 2) ./ res[:, 1])],
    rel_mae  = [mean(abs.(res[:, 1] .- res[:, 2]) ./ res[:, 1]), mean(abs.(res[:, 1] .- res[:, 3]) ./ res[:, 1]), mean(abs.(res[:, 1] .- res[:, 4]) ./ res[:, 1])],
    est      = ["Expected value","glm","ols"]
)

Row,rel_bias,rel_mse,rel_mae,est
,Float64,Float64,Float64,String
1,5.09603e-5,0.942724,0.000962992,Expected value
2,0.0539216,3598.31,0.061367,glm
3,0.0747812,6713.0,0.0831192,ols


In [17]:
DataFrame(
    rel_bias = [mean((res[:, 2] .- res[:, 3]) ./ res[:, 2]), mean((res[:, 2] .- res[:, 4]) ./ res[:, 2])],
    rel_mse  = [mean(((res[:, 2] .- res[:, 3]) .^ 2) ./ res[:, 2]), mean(((res[:, 2] .- res[:, 4]) .^ 2) ./ res[:, 2])],
    rel_mae  = [mean(abs.(res[:, 2] .- res[:, 3]) ./ res[:, 2]), mean(abs.(res[:, 2] .- res[:, 4]) ./ res[:, 2])],
    est      = ["glm","ols"]
)

Row,rel_bias,rel_mse,rel_mae,est
,Float64,Float64,Float64,String
1,0.0538758,3590.22,0.0612974,glm
2,0.0747362,6703.69,0.0830519,ols
